<a href="https://colab.research.google.com/github/aruaru0/llm-hands-on/blob/main/llm_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colabの使い方

1. [https://colab.google/](https://colab.google/)にアクセス→"New Notebook"または「新規作成」を選択
2. Googleアカウントでのログインを求められた場合はログインする
3. コードブロック（セル）にPythonコードを書いてShift-Enterで実行

## 練習（１）
とりあえず、"hello"と表示



In [ ]:
print("hello")

hello


# LLMハンズオン用コード

## 実行前の注意点
**実行前に必ず「ランタイム」→「ランタイムのタイプを変更」で<font color=red>"T4 GPU"</font>を選択しておくこと！**

# 1. 前準備 ==============

## ⭐️⭐️　今回は、デコーダーモデルを使って演習を行います

## ①必要なパッケージをインポートする
以下の２つのパッケージから、必要となる機能をインポートします
* transformerからAutoModelForCasualLMとAutoTokenizer
* torchとtorch.nn.functional

`as F`は、**Fという名前**で読み込むことを指定する記述方法です

<font color="blue">下記のコードを入力したら、コードセル左の実行ボタンをクリック、またはshift+enterでコードを実行してください。</font>

In [ ]:
# パッケージのインポート
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn.functional as F

## ②モデルを読み込む

モデルは、トークン化するための**tokenizer**と、デコーダー本体の**AutoModelForCausalLM**に分かれています。今回は、モデルは楽天のRakutenAI-2.0-miniを利用します。  

* tokenizerとmodelは一致していないとうまく動きません（<font color="red">トークナイザーで変換されるトークンIDとモデルのトークンIDが一致している必要があります</font>）
* llamaなどのモデルはHuggingface.ioのサイトにログインし認証する必要があるので注意

**学習済みモデル**を使いたいので`from_pretrained`（訓練済みの指示）をつけています。

また、モデルはdeviceをautoにしています（GPUがあれば自動的にGPUに転送されます）

In [ ]:
model_path = "Rakuten/RakutenAI-2.0-mini"
# トークナイザー（トークン化するためのライブラリ）
tokenizer = AutoTokenizer.from_pretrained(model_path)
# デコーダーモデルを読み込む
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="auto", device_map="auto")
# 評価モードへ
model.eval()

tokenizer_config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/755k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.07G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(48000, 2048)
    (layers): ModuleList(
      (0-21): 22 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((2048,), eps=1e-05)
  

# 2. 文字列とトークンIDの相互変換 ==============

ここでは、文字列がどのような「トークンIDになるのか？」を確認してみます

このために、文字列→トークンIDと、変換されたトークンIDから文字列への変換を行ってみます

## 1. 文字列からトークンID

* 先ほどロードしたtokenizerに文字列を入力するとトークンIDに変換されます

## 2. トークンIDから単語

* tokenizerには、トークンIDを文字列に直す関数も用意されています`convert_ids_to_tokens()`

※ トークンには特殊なものがあります。例えば`<s>`は文章の開始を表すトークンです（変換時に自動的に付加されます）


## ③文字列→トークンID

In [ ]:
text = "this is a pen."
input = tokenizer(text)
print(input)

{'input_ids': [1, 894, 349, 264, 4969, 28723], 'attention_mask': [1, 1, 1, 1, 1, 1]}


## ④トークンID→単語

In [ ]:
# tokenをデコードして文字列に直す
tokenizer.convert_ids_to_tokens(input['input_ids'])

['<s>', 'this', '▁is', '▁a', '▁pen', '.']

# 3. embedding（トークンのベクトル表現）を確認する  ==============

ここでは、トークンIDがベクトル表現になるのを確認してみます

ベクトル表現を取り出すには、modelのembedding（埋め込み層）の出力を取り出す必要があります。

modelから埋め込み層だけ取り出す関数が`get_input_embeddings()`として用意されていますのでこれを使います。

* 文字列をトークンIDに変換する
* 埋め込み層だけ取り出す
* トークンIDを埋め込み層に入力してベクトル表現を受け取る
* 受け取った内容を表示する（大きいので、ベクトルの形状と先頭の一部を確認する）

### ポイント
1. 今回使ったモデルでは、2048次元のベクトルに変換されます
2. 出力は（1, トークン数, ベクトルサイズ）の形式で格納されます

## ⑤ベクトル表現を取り出す

In [ ]:
text = "Hello, how are you?"
inputs = tokenizer(text, return_tensors="pt")

# 埋め込み層を取得
embedding_layer = model.get_input_embeddings()

# トークン ID を埋め込みベクトルに変換
input_ids = inputs["input_ids"].to(model.device)  # GPU/CPUのデバイスに合わせる
embedding_output = embedding_layer(input_ids)

# 埋め込みベクトルの形状を表示
print("入力トークンID ", input_ids)
print("入力トークン   ", tokenizer.convert_ids_to_tokens(input_ids[0]))
print("ベクトルの形状 ", embedding_output.shape)     # ベクトルの形状を確認(1, sequence_length, hidden_size)
print("ベクトルの中身 ", embedding_output[0,1,:100]) # helloのベクトルの先頭100個を表示

# 3. デコーダーを動かしてみる ==============
デコーダーモデルを実際に動かしてみます

* デコーダの生の出力を確認してみます
* 入力した文字列の次の文字が予測できていることを確認します。ここでは、予測結果の上位５つを表示してみます。

## ポイント
1. モデルをGPUに配置しているので、変換したトークンIDもGPUに移動させる必要があります（`to(model.device)`）
1. 予測スコアはモデルの出力の`logits`に格納されています
1. 最後のトークンの予測スコアだけ取り出すので末尾を取り出します（pythonでは-1で後ろから参照できます）
1. `softmax(..., dim=-1)`で行ごとに正規化が行われます
1. 出力の形状から、トークンIDの数が分かります。確認してみてください
1. `decode`を使うことで、トークンIDを文字に戻しています

## ⑥次の文字を予測してみる

In [ ]:
# 入力文
input_text = "日本で一番高い山は"
# トークン化
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)
# モデルでの予測
with torch.no_grad():
    # モデルを使用して出力を得る
    outputs = model(input_ids)
    # ログits（モデルの出力生のスコア）を取得
    logits = outputs.logits
    # 最後のトークンに対応するスコアを取り出す
    last_token_logits = logits[:, -1, :]
    # softmaxを適用して確率分布を得る
    softmax_probs = F.softmax(last_token_logits, dim=-1)

print(f"入力テキスト: {input_text} : {input_ids}")
print(f"出力の形状: {logits.shape}")

# 上位k個を取得
k = 5
top_k_values, top_k_indices = torch.topk(softmax_probs, k=5, dim=-1)

# 結果の表示
for prob, predicted_token_id in zip(top_k_values[0], top_k_indices[0]):
  predicted_token = tokenizer.decode(predicted_token_id, skip_special_tokens=True)
  print("-"*40)
  print(f"softmax確率: {prob*100:.2f}%")
  print(f"  予測されたトークンID: {predicted_token_id.item()}")
  print(f"  予測されたトークン: {predicted_token}")

## ⑦デコーダーを連続で動かしてみる

予測した文字列を追加しながら、連続で動かして文章が生成されていくのを確認してみます。
* 先ほどのプログラムをループに変えます
* `argmax`で一番スコアの高いものを選択するように変更します

In [ ]:
# 入力文
input_text = "昨日は雨でした。傘を"

# 5回ループする
for _ in range(5):
  # モデルでの予測
  with torch.no_grad():
      # トークン化
      input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

      # モデルを使用して出力を得る
      outputs = model(input_ids)

      # ログits（モデルの出力生のスコア）を取得
      logits = outputs.logits

      # 最後のトークンに対応するスコアを取り出す
      last_token_logits = logits[:, -1, :].cpu()

      # softmaxを適用して確率分布を得る
      softmax_probs = F.softmax(last_token_logits, dim=-1)

      # 予測されたトークンIDを取得
      predicted_token_id = torch.argmax(softmax_probs, dim=-1)

      predicted_token = tokenizer.decode(predicted_token_id, skip_special_tokens=True)
      print(predicted_token_id, tokenizer.eos_token_id)

      # 末尾に予測を足してループ
      input_text += predicted_token
      print(input_text)

# 4. Generateを使う例 ==============

ここまで自分で１文字つづ予測し追加していましたが、これを自動で行う`generate()`という関数があるのでこれを使ってみます。

`generate`の引数
* `max_new_tokens`　最大のトークン数
* `do_sample`　サンプル戦略を有効にする（trueの場合モデルは確率分布に基づいてランダムにトークンを選択）
* `temperature`　どの程度の確率で選ぶか（小さくするほど確率の高いものを選択するようになる）
* `top_p`　0.9の場合、最も確率が高い90%のトークンの中から次を選択する
* `repetition_penalty`　同じトークンを繰り返し生成することを抑制するためのペナルティ値（大きくするほど繰り返しが抑制される）

## ⑧デコーダの一般的な使い方

In [ ]:
inputs = tokenizer("この物語は、", return_tensors="pt").to(model.device)
with torch.no_grad():
    tokens = model.generate(
        **inputs,
        max_new_tokens=32,
        do_sample=True, # サンプリング戦略を有効にする（tempertureなどが有効になる）
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.05,
        pad_token_id=tokenizer.pad_token_id,
    )

output = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(output)